# finding-missing-person-using-AI

In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                shear_range=0.2,
                                rotation_range=180,
                                horizontal_flip=True,
                                zoom_range=0.2)
test_datagen=ImageDataGenerator(rescale=1./255)

In [3]:
x_train=train_datagen.flow_from_directory(r'C:\Users\Harsha reddy\Downloads\Dataset\Dataset\trainset',
                                          target_size=(64,64),batch_size=32,
                                          class_mode='binary')

Found 240 images belonging to 2 classes.


In [4]:
x_test=train_datagen.flow_from_directory(r'C:\Users\Harsha reddy\Downloads\Dataset\Dataset\testset', 
                                         target_size=(64,64),batch_size=32,
                                         class_mode='binary')

Found 60 images belonging to 2 classes.


In [5]:
print(x_train.class_indices)

{'Found Missing': 0, 'Normal': 1}


adding models and layers

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten

In [7]:
model=Sequential()

In [8]:
model.add(Conv2D( 32,3,3,input_shape=(64,64,3),activation='relu'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
model.add(MaxPool2D(pool_size=(2,2)))

In [10]:
model.add(Flatten())

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 21, 21, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0         
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.add(Dense(units=128,kernel_initializer='random_uniform',activation='relu'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [13]:
model.add(Dense(units=128,kernel_initializer='random_uniform',activation='relu'))

In [14]:
model.add(Dense(units=1,activation='sigmoid',kernel_initializer='random_uniform'))

In [15]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [16]:
model.fit_generator(x_train , steps_per_epoch=8 , epochs=300, validation_data = x_test, validation_steps = 2 )

Epoch 1/300
8/8 [==============================] - 19s 2s/step - loss: 0.6607 - acc: 0.6667 - val_loss: 0.6428 - val_acc: 0.6667
Epoch 2/300
8/8 [==============================] - 6s 769ms/step - loss: 0.6482 - acc: 0.6667 - val_loss: 0.6467 - val_acc: 0.6667
Epoch 3/300
8/8 [==============================] - 7s 880ms/step - loss: 0.6398 - acc: 0.6667 - val_loss: 0.6519 - val_acc: 0.6667
Epoch 4/300
8/8 [==============================] - 7s 852ms/step - loss: 0.6354 - acc: 0.6667 - val_loss: 0.6613 - val_acc: 0.6667
Epoch 5/300
8/8 [==============================] - 11s 1s/step - loss: 0.6202 - acc: 0.6667 - val_loss: 0.6618 - val_acc: 0.6667
Epoch 6/300
8/8 [==============================] - 9s 1s/step - loss: 0.6154 - acc: 0.6667 - val_loss: 0.6702 - val_acc: 0.6667
Epoch 7/300
8/8 [==============================] - 7s 896ms/step - loss: 0.6192 - acc: 0.6667 - val_loss: 0.6744 - val_acc: 0.6667
Epoch 8/300
8/8 [==============================] - 8s 956ms/step - loss: 0.6185 - acc: 0.6

save the model

In [17]:
model.save('Missing_1.h5')

In [18]:
import cv2
import numpy as np
import smtplib
from keras.preprocessing import image
import tensorflow as tf
import os
name = ["Found Missing","Normal"]

In [19]:
model = tf.keras.models.load_model('Missing_1.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [26]:
img = image.load_img(r"C:\Users\Harsha reddy\Pictures\image6.jpg",target_size = (64,64))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)

In [27]:
pred=model.predict_classes(x)

In [28]:
pred[0][0]

0

In [29]:
index=['Found missing','normal']

In [30]:
a = index[pred[0][0]]

In [31]:
a

'Found missing'

# Video Analysis

In [ ]:
import cv2
import numpy as np
from keras.preprocessing import image
import tensorflow as tf
from twilio.rest import Client

model = tf.keras.models.load_model('Missing_1.h5')
video = cv2.VideoCapture(0)
name = ["Found Missing","Normal"]

while(True):
    success, frame = video.read()
    cv2.imwrite("image.jpg",frame)
    img = image.load_img("image.jpg",target_size = (64,64))
    x = image.img_to_array(img)
    x = np.expand_dims(x,axis = 0)
    pred = model.predict_classes(x)
    p = pred[0][0]
    print(p)
    cv2.putText(frame, "predicted class = "+str(name[p]),(100,100),
               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1)
    
    if pred[0][0]==0:
        account_sid='AC234d058ee261c91dbd5e3a736155fed0'
        auth_token='b322aed33ef942adbb136d6eb20fd8c5'
        client=Client(account_sid,auth_token)
        message=client.messages.create(
        to="+919390537480",
        from_= "+17725714637",
        body=" Found the Missing at 17.3984 N, 78.5583 E"
        )
        print(message.sid)
        print("Found Missing")
        print('SMS Sent')
    else:
        print("Normal")
    
    cv2.imshow("frame",frame)

    if cv2.waitKey(1) & 0xFF == ord('a'):
        break
    
    
    
video.release()
cv2.destroyALLWindow()